In [1]:
from kafka import KafkaConsumer
from kafka import TopicPartition
from datetime import datetime
import pandas as pd
from cassandra.cluster import Cluster
import json
from botocore.vendored import requests
import findspark
findspark.init()
import pyspark.sql 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
import pandas as pd
import time
from cassandra.cluster import Cluster
import numpy as np
from pyspark.sql.types import *

In [2]:
# #------------------------------- SPARK CONTEXT -------------------------------
sc = SparkContext('local')
spark = SparkSession(sc)
spark = SparkSession.builder.appName('CC_Fraud').getOrCreate()


In [3]:
def MachineLearningModelLoading():
	path = r"C:\Users\saritajoshi\Documents\Repos\Credit-Card-Fraud-Detection-Spark\model"
	newmodel = PipelineModel.load(path)
	return newmodel
	
newmodel = MachineLearningModelLoading()


In [4]:
# feature_columns = [col for col in df.columns if col != "is_fraud"]
# assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [5]:
def predicting_status(data_all,newmodel):
	select_cols = ['amt', 'lat', 'long', 'city_pop', 'unix_time','merch_lat', 'merch_long','merchant_label','category_label', 'gender_label', 'job_label'] #all numeric
	data = data_all.select(select_cols)
	pred1 = newmodel.transform(data)
	return pred1.select("prediction")

# df = spark.read.json(sc.parallelize([dd]))
# res = predicting_status(df, newmodel)

In [6]:
cluster = Cluster()
session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")

#session.execute("DROP TABLE testing")
# session.execute("CREATE TABLE IF NOT EXISTS testing (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount double, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")


In [7]:
topic_name = 'transaction_data'
keyspace = 'bigdata'
table = 'transaction_data'

In [8]:
consumer = KafkaConsumer(topic_name, group_id=None, bootstrap_servers=['localhost:9092'])
for msg in consumer:
	try:
		print('msg', msg)
		msg=msg.value.decode("utf-8")
		output = json.loads(msg)
		df = spark.read.json(sc.parallelize([msg]))
		status = predicting_status(df,newmodel)

		status = int(status.collect()[0][0])
		output['is_fraud_prediction'] = status
		output['inserted_at'] = int(time.time() * 1000)

		insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json.dumps(output)}';"
		session.execute(insert_query)
	except Exception as e:
		print('Error in transaction:', e)


msg ConsumerRecord(topic='transaction_data', partition=0, offset=2726, timestamp=1715387551529, timestamp_type=0, key=b'null', value=b'{"trans_date_trans_time": "2020-12-06 22:09:51", "cc_num": 4451438912406400, "merchant": "fraud_Hirthe-Beier", "category": "health_fitness", "amt": 85.81, "first": "Cheryl", "last": "Morgan", "gender": "F", "street": "7535 Amanda Manors Suite 293", "city": "Norman", "state": "AR", "zip": 71960, "lat": 34.4596, "long": -93.6743, "city_pop": 1383, "job": "Manufacturing engineer", "dob": "1999-09-11", "trans_num": "ee533fe651486dd4f7e1e1097b4d7bb9", "unix_time": 1386367791, "merch_lat": 34.770419, "merch_long": -93.430473, "is_fraud": 0, "merchant_label": 260, "category_label": 5, "gender_label": 0, "job_label": 283}', headers=[], checksum=None, serialized_key_size=4, serialized_value_size=621, serialized_header_size=-1)
msg ConsumerRecord(topic='transaction_data', partition=0, offset=2727, timestamp=1715387551530, timestamp_type=0, key=b'null', value=b'{"

In [ ]:
consumer.close()
spark.stop()

In [ ]:
# insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json.dumps(output)}';"
# session.execute(insert_query)